# Thrombectomy pilot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
from xgboost import XGBClassifier
import seaborn as sns


In [ ]:
data = pd.read_csv('./data/complete_ml_data.csv')

In [ ]:
X_fields = [
        'prior_disability',
        'stroke_severity',
        'onset_to_thrombolysis',
        'age',
        'precise_onset_known',
        'any_afib_diagnosis',
        'onset_to_thrombectomy']



In [ ]:
# In data where onset_to_thrombolysis is empty set to 99999
data['onset_to_thrombolysis'] = data['onset_to_thrombolysis'].fillna(99999)
data['arrival_to_thrombectomy_time'] = data['arrival_to_thrombectomy_time'].fillna(99999)
# Limit to stroke severity > 5
data = data[data['stroke_severity'] > 5]
# Where arrival to thrombectomy time is present calculate onset to thrombectomy
data['onset_to_thrombectomy'] = data['arrival_to_thrombectomy_time'] + data['onset_to_arrival_time']
# Set max onset to thrombectomy to 99999
data['onset_to_thrombectomy'] = data['onset_to_thrombectomy'].apply(lambda x: min(x, 99999))
# Limit to patients arriving 6 hours after onset
data = data[data['onset_to_arrival_time'] <= 360]
# Limit is ischaemic stroke only
data = data[data['infarction'] == 1]
# Exclude where thrombectomy time >480 and < 99999 (avoid using late thrombectomy)
data = data[(data['onset_to_thrombectomy'] <= 480) | (data['onset_to_thrombectomy'] == 99999)]


In [ ]:
X = data[X_fields]
y = data['discharge_disability'] <= 2

# Split 80:20
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train)


Test model

In [ ]:
# Fit the model
model = XGBClassifier(verbosity=0, seed=42)
model.fit(X_train, y_train)
# Get ROC AUC
from sklearn.metrics import roc_auc_score
y_pred = model.predict(X_test)
y_probs = model.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_probs)
print(f"ROC AUC: {roc_auc_score(y_test, y_probs):.3f}")


In [ ]:
# Plot a seaborn regression plot with confidence limits
sns.regplot(x=thrombectomy_time, y=thrombectomy_effect, ci=95, 
            scatter_kws={'alpha':0.15}, line_kws={'color': 'red'})
plt.xlabel('Onset to thrombectomy time (minutes)')
plt.ylabel('Effect on log odds of discharge mRS 0-2')
# Add aline at y=0
plt.axhline(y=0, color='black', linestyle='--')
# Add p value
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(thrombectomy_time, thrombectomy_effect)

# Get intercepts at x and y axis
x_intercept = -intercept/slope
y_intercept = intercept

# Add r-shared, p value, and inercepts to plot
textstr = f'r² = {r_value**2:.3f}\np (slope) = {p_value:.5f}\nX intercept: {
    x_intercept:.0f} mins\nY intercept: {y_intercept:.2f}'
plt.text(330, 2.0, textstr)

# save the plot
plt.savefig('thrombectomy.png', dpi=300)
plt.show()